# ADS 509 Assignment 2.1: Tokenization, Normalization, Descriptive Statistics 

This notebook holds Assignment 2.1 for Module 2 in ADS 509, Applied Text Mining. Work through this notebook, writing code and answering questions where required. 

In the previous assignment you put together Twitter data and lyrics data on two artists. In this assignment we explore some of the textual features of those data sets. If, for some reason, you did not complete that previous assignment, data to use for this assignment can be found in the assignment materials section of Blackboard. 

This assignment asks you to write a short function to calculate some descriptive statistics on a piece of text. Then you are asked to find some interesting and unique statistics on your corpora. 


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


In [2]:
pip install emoji

     |████████████████████████████████| 240 kB 5.2 MB/s eta 0:00:01
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234906 sha256=ea9b5b22619f6a8cebad9dd64f68ce7a958cb3faf49e78e2619c3a8c2380010e
  Stored in directory: /Users/georgegarcia/Library/Caches/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji
Note: you may need to restart the kernel to use updated packages.


In [72]:
import os
import re
import emoji
import pandas as pd
import numpy as np
import nltk

from collections import Counter, defaultdict
from string import punctuation
nltk.download('stopwords')

sw = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/georgegarcia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
# Add any additional import statements you need here
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, \ 
    compile_infix_regex, compile_suffix_regex



In [73]:
# change `data_location` to the location of the folder on your machine.
data_location = "/users/georgegarcia/Desktop/509/"

# These subfolders should still work if you correctly stored the 
# data from the Module 1 assignment
twitter_folder = "twitter/"
lyrics_folder = "lyrics/"

In [74]:
def descriptive_stats(tokens, num_tokens = 5, verbose=True) :
    """
        Given a list of tokens, print number of tokens, number of unique tokens, 
        number of characters, lexical diversity (https://en.wikipedia.org/wiki/Lexical_diversity), 
        and num_tokens most common tokens. Return a list with the number of tokens, number
        of unique tokens, lexical diversity, and number of characters. 
    
    """

    # Fill in the correct values here. 
    #using len to count the tokens 
    num_tokens = len(tokens)
    #using set and len to count the unique tokens 
    num_unique_tokens = len(set(tokens))
    #finding the diversity 
    lexical_diversity = num_unique_tokens / num_tokens
    #count the number of characters 
    num_characters = sum(len(word) for word in tokens)
    
    if verbose :        
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
    
        # print the five most common tokens
        
    return([num_tokens, num_unique_tokens,
            lexical_diversity,
            num_characters])
    

In [75]:
text = """here is some example text with other example text here in this text""".split()
assert(descriptive_stats(text, verbose=True)[0] == 13)
assert(descriptive_stats(text, verbose=False)[1] == 9)
assert(abs(descriptive_stats(text, verbose=False)[2] - 0.69) < 0.02)
assert(descriptive_stats(text, verbose=False)[3] == 55)


There are 13 tokens in the data.
There are 9 unique tokens in the data.
There are 55 characters in the data.
The lexical diversity is 0.692 in the data.


Q: Why is it beneficial to use assertion statements in your code? 

A: <-- assertion statements are usefull for self-checking code and documentation. With assertion statements it can debugging the a section of the code or sharing--> 

## Data Input

Now read in each of the corpora. For the lyrics data, it may be convenient to store the entire contents of the file to make it easier to inspect the titles individually, as you'll do in the last part of the assignment. In the solution, I stored the lyrics data in a dictionary with two dimensions of keys: artist and song. The value was the file contents. A data frame would work equally well. 

For the Twitter data, we only need the description field for this assignment. Feel free all the descriptions read it into a data structure. In the solution, I stored the descriptions as a dictionary of lists, with the key being the artist. 




In [49]:
# Read in the lyrics data
lyrics_data = pd.read_csv("lyrics.csv")

# create an empty dictionary to store the lyrics data
lyrics_dict = {}

# iterate over the rows of the Dataframe
for _, row in lyrics_data.iterrows():
    artist = row["artist"]
    song = row["song"]
    lyrics = row["lyrics"]

    # if the artist does not already exist in the dictionary, create a new key for them
    if artist not in lyrics_dict:
        lyrics_dict[artist] = {}

    # add the song and lyrics to the artist's dictionary
    lyrics_dict[artist][song] = lyrics

print(lyrics_dict)

FileNotFoundError: [Errno 2] No such file or directory: 'twitter_folder'

In [77]:
cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory
print("Files in %r: %s" % (cwd, files))

Files in '/Users/georgegarcia/Documents/GitHub/BVBeda': ['lyrics', 'twitter', '.DS_Store', 'API and Scrape.ipynb', 'API and Scrape - Solutions.ipynb', 'README.md', '.gitignore', '.ipynb_checkpoints', 'api_keys.py', 'Lyrics and Description EDA.ipynb', '.git', 'gitignore.txt', 'Enhancing_BERT_for_lexical_normalisation_WNUT2019_proceeding-5.pdf']


In [94]:
# Read in the twitter data
#twitter_data = pd.read_csv("/Users/georgegarcia/Documents/GitHub/BVBeda/twitter/robynkonichiwa_follower_data.txt"
T_file = "/Users/georgegarcia/Documents/GitHub/BVBeda/twitter/robynkonichiwa_follower_data.txt"
posts_df = pd.read_csv(T_file , skiprows=[23721])
subred_file = "/Users/georgegarcia/Documents/GitHub/BVBeda/twitter/robynkonichiwa_followers.txt"
subred_df = pd.read_csv(subred_file).set_index(['subreddit'],skiprows=[5])
df = posts_df.join(subred_df, on='subreddit')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 5, saw 2


In [ ]:
# create an empty dictionary to store the twitter data
twitter_dict = {}

# iterate over the rows of the Dataframe
for _, row in twitter_data.iterrows():
    artist = row["artist"]
    descriptions = row["descriptions"]

    # if the artist does not already exist in the dictionary, create a new key for them
    if artist not in twitter_dict:
        twitter_dict[artist] = []

    # add the description to the artist's dictionary
    twitter_dict[artist].append(twitter_dict)
print(twitter_dict)

## Data Cleaning

Now clean and tokenize your data. Remove punctuation chacters (available in the `punctuation` object in the `string` library), split on whitespace, fold to lowercase, and remove stopwords. Store your cleaned data, which must be accessible as an interable for `descriptive_stats`, in new objects or in new columns in your data frame. 



In [ ]:
punctuation = set(punctuation) # speeds up comparison

In [70]:
# create your clean twitter data here

#creating the def for stop words from sw datafram 

pipeline = [str.lower, tokenize, remove_stop, split()]
def prepare(text, pipeline):
tokens = text)
for transform in pipeline:
tokens = transform(tokens)
return tokens

#apply the prepare function to the teittter_dict 
clean_t = prepare(twitter_dict)

IndentationError: expected an indented block (<ipython-input-70-f6d4b0e9b13c>, line 7)

In [ ]:
# create your clean lyrics data here
#apply the prepare function to the lyrics_dict 
clean_l = prepare(lyrics_dict)

## Basic Descriptive Statistics

Call your `descriptive_stats` function on both your lyrics data and your twitter data and for both artists (four total calls). 

In [ ]:
# calls to descriptive_stats twitter
descriptive_stats(clean_t)

In [ ]:
# calls to descriptive_stats lyrics
descriptive_stats(clean_l)

Q: How do you think the "top 5 words" would be different if we left stopwords in the data? 

A: If we left the stop words in the data the top 5 would be completly different it would be the connceting words like to and the 

---

Q: What were your prior beliefs about the lexical diversity between the artists? Does the difference (or lack thereof) in lexical diversity between the artists conform to your prior beliefs? 

A: I thought that cher whould have a more lexical diversity becasue even my foreign parents know who cher is but they dont know the other guy 



## Specialty Statistics

The descriptive statistics we have calculated are quite generic. You will now calculate a handful of statistics tailored to these data.

1. Ten most common emojis by artist in the twitter descriptions.
1. Ten most common hashtags by artist in the twitter descriptions.
1. Five most common words in song titles by artist. 
1. For each artist, a histogram of song lengths (in terms of number of tokens) 

We can use the `emoji` library to help us identify emojis and you have been given a function to help you.


In [6]:
assert(emoji.is_emoji("❤️"))
assert(not emoji.is_emoji(":-)"))

### Emojis 😁

What are the ten most common emojis by artist in the twitter descriptions? 


In [ ]:
# Your code here

### Hashtags

What are the ten most common hashtags by artist in the twitter descriptions? 


### Song Titles

What are the five most common words in song titles by artist? The song titles should be on the first line of the lyrics pages, so if you have kept the raw file contents around, you will not need to re-read the data.


In [ ]:
# Your code here

### Song Lengths

For each artist, a histogram of song lengths (in terms of number of tokens). If you put the song lengths in a data frame with an artist column, matplotlib will make the plotting quite easy. An example is given to help you out. 


In [ ]:
num_replicates = 1000

df = pd.DataFrame({
    "artist" : ['Artist 1'] * num_replicates + ['Artist 2']*num_replicates,
    "length" : np.concatenate((np.random.poisson(125,num_replicates),np.random.poisson(150,num_replicates)))
})

df.groupby('artist')['length'].plot(kind="hist",density=True,alpha=0.5,legend=True)

Since the lyrics may be stored with carriage returns or tabs, it may be useful to have a function that can collapse whitespace, using regular expressions, and be used for splitting. 

Q: What does the regular expression `'\s+'` match on? 

A: 


In [ ]:
collapse_whitespace = re.compile(r'\s+')

def tokenize_lyrics(lyric) : 
    """strip and split on whitespace"""
    return([item.lower() for item in collapse_whitespace.split(lyric)])

In [ ]:
# Your lyric length comparison chart here. 